In [9]:
import base64
import requests
from bs4 import BeautifulSoup as bs
from secrets import Secret 
import json

class MakeHtml:
    def __init__(self):
        secret = Secret()
        # confluence OAuth AppId and Secrets
        self.confluenceAppId = secret.confluenceAppId
        self.confluenceSecretkey = secret.confluenceSecretkey
        
        # redirectUrl
        self.redirectUrl = 'https://ykarma1996.tistory.com'
        
        # imgBB api key
        self.imgApiKey = secret.apiKeyImgbb
        # user's confluence space baseurl
        self.baseUrl = "https://api.atlassian.com/ex/confluence/"
        # header for api requests
        self.headers = {"Authorization": "Bearer {token}", 'Accept': 'application/json'}
        # htmlBodySoup
        self.htmlSoup = ""
    
    def oauthMakeLink(self):
        makeLinkUrl = 'https://auth.atlassian.com/authorize'
        params = {
            'audience': 'api.atlassian.com',
            'client_id': self.confluenceAppId,
            'scope': 'read:confluence-space.summary read:confluence-content.all read:confluence-content.summary read:confluence-props search:confluence',
            'redirect_uri': self.redirectUrl,
            'state': 'random',
            'response_type': 'code',
            'prompt': 'consent'
        }
        res = requests.get(makeLinkUrl, params=params)
        if (res.status_code == 200):
            print (res.url)
        else:
            print ('Made Url is not available')    
    
    # fill api access information and check api can speak
    def loginProcess(self, authCode, userName):
        getTokenUrl = 'https://auth.atlassian.com/oauth/token'
        headers = self.headers
        headers['Content-Type'] = 'application/json'
        data = {
            'grant_type': 'authorization_code',
            'client_id': self.confluenceAppId,
            'client_secret': self.confluenceSecretkey,
            'code': authCode,
            'redirect_uri': self.redirectUrl
        }
        authRes = requests.post(getTokenUrl, data=json.dumps(data), headers=headers)
        if (authRes.status_code == 200):
            token = authRes.json()['access_token']
            print ('Success to get Token!')
        else:
            print ('Auth Fail!')
            return False
        
        self.headers["Authorization"] = "Bearer {}".format(token)
        
        getBaseUrl = 'https://api.atlassian.com/oauth/token/accessible-resources'
        getBaseRes = requests.get(getBaseUrl, headers=self.headers)
        if (getBaseRes.status_code == 200):
            baseId = getBaseRes.json()[0]['id']
            self.baseUrl = 'https://api.atlassian.com/ex/confluence/{}'.format(baseId)
            print ("Success get Id of domain!")
        else:
            print (getBaseRes.status_code)
            return False
        
        # check api
        testUrl = "{}/rest/api/space".format(self.baseUrl)
        print (testUrl)
        testRes = requests.get(getBaseUrl, headers=self.headers)
        if (testRes.status_code == 200):
            print ('Success login!')
        else:
            print (testRes.status_code)
            return False
        
        
    
    # get space list from 
    def getSpaceList(self):
        spaceListUrl = "{}/rest/api/space".format(self.baseUrl)
        res = requests.get(spaceListUrl, headers=self.headers).json()
        
        # spaceListDict {"sample": {"id":1234, "key": "SPACE1", "selfUrl": "https://example.com"}}
        spaces = {}
        for item in res['results']:
            spaces[item['name']] = {}
            spaces[item['name']]['id'] = item['id']
            spaces[item['name']]['key'] = item['key']
        print (spaces)
        return spaces

    def getContentList(self, spaceKey):
        params = {"spaceKey": spaceKey}
        contentListUrl = "{}/rest/api/content".format(self.baseUrl)
        res = requests.get(contentListUrl, params=params, headers=self.headers).json()
        
        # contentListDict {"id(num)": "title"}
        contents = {}
        for item in res['results']:
            contents[item['id']] = item['title']
            
        print (contents)
        return contents
    
    def getCententHtml(self, contentId):
        viewType = "view"
        params = {"expand": "body.{}".format(viewType)}
        contentUrl = "{}/rest/api/content/{}".format(self.baseUrl, contentId)
        res = requests.get(contentUrl, params=params, headers=self.headers).json()
        htmlBody = res['body'][viewType]['value']
        self.htmlSoup = bs(htmlBody, 'html.parser')
        self.rebuildFormat()
        self.rebuildImgStore(contentUrl)
        
    def rebuildFormat(self):
        # replace all 'th' tag to 'tr' tag
        for th in self.htmlSoup.findAll('th'):
            th.name = 'td'
        
        # beautify table view
        for table in self.htmlSoup.findAll('table'):
            table.attrs['data-ke-style'] = 'style12'
           
        # append <code> tag inside <pre> tag for beautify code block
        for pre in self.htmlSoup.findAll('pre', class_='syntaxhighlighter-pre'):
            newTag = self.htmlSoup.new_tag('code')
            newTag.string = pre.string
            pre.string = ""
            pre.append(newTag)

            
    # upload image to imgbb and return image's url
    def uploadImg(self, imgBin):
        imgUpload = "https://api.imgbb.com/1/upload"
        params = {'key': self.imgApiKey}
        imgBase64 = base64.b64encode(imgBin).decode('ascii')
        data = {'image': imgBase64}
        res = requests.post(imgUpload, params=params, data=data) #.json()
        return res['data']['url']
        
    # replace <img> tag for public viwer
    def rebuildImgStore(self, contentUrl):
        for img in self.htmlSoup.findAll('img'):
            print (img)
            imgSrc = img.attrs['src']
            imgSrc = '{}/child/attachment'.format(contentUrl)
            imgSrc = '{}/download/attachments/18284545/image-20210326-085440.png?version=1&modificationDate=1616748901731&cacheVersion=1&api=v2'.format(self.baseUrl)
            print (self.headers)
            res = requests.get(imgSrc, headers=self.headers)
            print (imgSrc)
            print (res.text)
            
            imgUrl = self.uploadImg(res.content)
            # replace <img> tag's src and delete not use attrs
            img.attrs['src'] = imgUrl
            img.attrs['data-image-src'] = ""
            img.attrs['data-base-url'] = ""
        print (self.htmlSoup)
    
    def saveHtmlFile(self, fileName):
        with open("{}.html".format(fileName), "w", -1, "utf-8") as f:
            f.write(str(self.htmlSoup))

            
def main():
    make1 = MakeHtml()
    make1.oauthMakeLink()
    authCode = input('Auth Code : ')
    userName = "geunsam2"
    make1.loginProcess(authCode, userName)
    make1.getSpaceList()

    spaceName = input ('input space key : ')
    spaceName = 'GEUNSAM2'
    lists = make1.getContentList(spaceName)

    contendId = input('input content ID : ')
    contendId = '18284545'
    make1.getCententHtml(contendId)
    return None

if __name__ == "__main__":
    main()


https://auth.atlassian.com/login?state=g6Fo2SAweEx3VXItM0llQkkzanE0bno3RkRmdC0xbXZ1S20tbqN0aWTZIFpXU2lLLThGR0tSTy1DQ1JVMHdfckdEX2hvT1BNRU5Fo2NpZNkgQ254ZVZtTWRaQngyeWE0N01CUjhhMGN6UW1Pa3dtOUo&client=CnxeVmMdZBx2ya47MBR8a0czQmOkwm9J&protocol=oauth2&prompt=consent&audience=api.atlassian.com&scope=read%3Aconfluence-space.summary%20read%3Aconfluence-content.all%20read%3Aconfluence-content.summary%20read%3Aconfluence-props%20search%3Aconfluence&redirect_uri=https%3A%2F%2Fykarma1996.tistory.com&response_type=code
Auth Code : Eh8ML0WohYwQuxxl
Success to get Token!
Success get Id of domain!
https://api.atlassian.com/ex/confluence/85dd2495-82c7-4a5f-818f-d5b05d30a806/rest/api/space
Success login!
{'geunsam2': {'id': 229377, 'key': 'GEUNSAM2'}, 'Project: gs': {'id': 14745699, 'key': 'GS'}, 'testspace': {'id': 7962636, 'key': 'TES'}}
input space key : 1
{'33156': 'CNI와 Docker', '163844': 'Calico?Weave? CNI에 관하여', '229378': 'geunsam2', '884816': 'CNI 동작에 관하여', '1933313': 'Ingress와 LoadBalancer', '7

TypeError: 'Response' object is not subscriptable